In [1]:
import numpy as np
import tensorflow as tf
# tf.enable_eager_execution()

In [2]:
tf.__version__

'1.12.0'

In [3]:
sess = tf.InteractiveSession()

In [4]:
for i in tf.data.Dataset.range(7).take(2):
    print(i)

RuntimeError: dataset.__iter__() is only supported when eager execution is enabled.

In [ ]:
for i in tf.data.Dataset.range(7).take(2):
    print(i)

In [34]:
a = tf.data.Dataset.range(7).window(3, 2, 1, True)
a

<WindowDataset shapes: <tensorflow.python.data.ops.dataset_ops._NestedDatasetComponent object at 0x7f38f0604a90>, types: <tensorflow.python.data.ops.dataset_ops._NestedDatasetComponent object at 0x7f38f0604a90>>

In [5]:
# for i in tf.data.Dataset.range(7).window(3, 2, 1, True):
#     print(i)

In [103]:
'''sliding window 搞定了!'''
# window: NestedDatasetComponent is trouble some. Need to be flattened by flat_map
# flat_map(lambda x: x.batch(2)): flatten the result and make it ready to output
ds = tf.data.Dataset.range(7).shuffle(buffer_size=5).window(2, 1, 1, True).flat_map(lambda x: x.batch(2)).batch(5)
# stride: 决定[0, 1]这种list内的距离
# shift:  决定[0, ..] [1, ..]这种list间的距离.


it = ds.make_one_shot_iterator()
data = it.get_next()

In [104]:
sess.run(data)

array([[0, 3],
       [3, 2],
       [2, 1],
       [1, 4],
       [4, 5]])

In [20]:
a = tf.data.Dataset.range(7).window(3, 2, 1, True)

iterator = a.make_one_shot_iterator()  
next = iterator.get_next()

TypeError: Expected DataType for argument 'output_types' not <tensorflow.python.data.ops.dataset_ops._NestedDatasetComponent object at 0x7f1304411748>.

In [14]:
sess.run(next)

6

## reinitializableI iterator 
**重新初始化的迭代器**

### 循环交替Trian & Evaluate
reinitializableI iterator 在这里并不适用：

In [9]:
# Define training and validation datasets with the same structure.
training_dataset = tf.data.Dataset.range(100)
validation_dataset = tf.data.Dataset.range(50)

# A reinitializable iterator is defined by its structure. We could use the
# `output_types` and `output_shapes` properties of either `training_dataset`
# or `validation_dataset` here, because they are compatible.
iterator = tf.data.Iterator.from_structure(training_dataset.output_types,
                                  training_dataset.output_shapes)

next_element = iterator.get_next()


training_init_op = iterator.make_initializer(training_dataset)
validation_init_op = iterator.make_initializer(validation_dataset)

with tf.Session() as sess:
   # Run 20 epochs in which the training dataset is traversed, followed by the
   # validation dataset.

    for i in range(2):
        print("#########################  epoch", i)
        # Initialize an iterator over the training dataset.
        # 这样init的话，下个epoch的时候，有init，从头开始数了...
        # 又不能放在epoch循环外面，因为这样的话，这个init会被下面的valid_init覆盖掉。。
        sess.run(training_init_op)
        for _ in range(10):
            nel = sess.run(next_element)
            print("train: ", type(nel), nel)
        
        sess.run(validation_init_op)
        print('evaluation start:')
        # Initialize an iterator over the validation dataset.
        for _ in range(5):
            nel = sess.run(next_element)
            print("valid: ", type(nel), nel)

#########################  epoch 0
train:  <class 'numpy.int64'> 0
train:  <class 'numpy.int64'> 1
train:  <class 'numpy.int64'> 2
train:  <class 'numpy.int64'> 3
train:  <class 'numpy.int64'> 4
train:  <class 'numpy.int64'> 5
train:  <class 'numpy.int64'> 6
train:  <class 'numpy.int64'> 7
train:  <class 'numpy.int64'> 8
train:  <class 'numpy.int64'> 9
evaluation start:
valid:  <class 'numpy.int64'> 0
valid:  <class 'numpy.int64'> 1
valid:  <class 'numpy.int64'> 2
valid:  <class 'numpy.int64'> 3
valid:  <class 'numpy.int64'> 4
#########################  epoch 1
train:  <class 'numpy.int64'> 0
train:  <class 'numpy.int64'> 1
train:  <class 'numpy.int64'> 2
train:  <class 'numpy.int64'> 3
train:  <class 'numpy.int64'> 4
train:  <class 'numpy.int64'> 5
train:  <class 'numpy.int64'> 6
train:  <class 'numpy.int64'> 7
train:  <class 'numpy.int64'> 8
train:  <class 'numpy.int64'> 9
evaluation start:
valid:  <class 'numpy.int64'> 0
valid:  <class 'numpy.int64'> 1
valid:  <class 'numpy.int64'> 

### 所有Train结束后Eval
reinitializableI iterator很适用

In [11]:
# Define training and validation datasets with the same structure.
training_dataset = tf.data.Dataset.range(100)
validation_dataset = tf.data.Dataset.range(50)

# A reinitializable iterator is defined by its structure. We could use the
# `output_types` and `output_shapes` properties of either `training_dataset`
# or `validation_dataset` here, because they are compatible.
iterator = tf.data.Iterator.from_structure(training_dataset.output_types,
                                  training_dataset.output_shapes)

next_element = iterator.get_next()


training_init_op = iterator.make_initializer(training_dataset)
validation_init_op = iterator.make_initializer(validation_dataset)

with tf.Session() as sess:
    # train两个epoch，结束后再eval.
    
    # Initialize an iterator over the training dataset.
    sess.run(training_init_op)
    for i in range(2):
        print("#########################  epoch", i)
        for _ in range(10):
            nel = sess.run(next_element)
            print("train: ", type(nel), nel)
        
    sess.run(validation_init_op)
    print('evaluation start:')
    # Initialize an iterator over the validation dataset.
    for _ in range(8):
        nel = sess.run(next_element)
        print("valid: ", type(nel), nel)

#########################  epoch 0
train:  <class 'numpy.int64'> 0
train:  <class 'numpy.int64'> 1
train:  <class 'numpy.int64'> 2
train:  <class 'numpy.int64'> 3
train:  <class 'numpy.int64'> 4
train:  <class 'numpy.int64'> 5
train:  <class 'numpy.int64'> 6
train:  <class 'numpy.int64'> 7
train:  <class 'numpy.int64'> 8
train:  <class 'numpy.int64'> 9
#########################  epoch 1
train:  <class 'numpy.int64'> 10
train:  <class 'numpy.int64'> 11
train:  <class 'numpy.int64'> 12
train:  <class 'numpy.int64'> 13
train:  <class 'numpy.int64'> 14
train:  <class 'numpy.int64'> 15
train:  <class 'numpy.int64'> 16
train:  <class 'numpy.int64'> 17
train:  <class 'numpy.int64'> 18
train:  <class 'numpy.int64'> 19
evaluation start:
valid:  <class 'numpy.int64'> 0
valid:  <class 'numpy.int64'> 1
valid:  <class 'numpy.int64'> 2
valid:  <class 'numpy.int64'> 3
valid:  <class 'numpy.int64'> 4
valid:  <class 'numpy.int64'> 5
valid:  <class 'numpy.int64'> 6
valid:  <class 'numpy.int64'> 7


## feedable iterator
**可馈送的迭代器**

In [12]:
# Define training and validation datasets with the same structure.
training_dataset = tf.data.Dataset.range(10000000).repeat(2)
validation_dataset = tf.data.Dataset.range(5000000).repeat(2)

# A feedable iterator is defined by a handle placeholder and its structure. We
# could use the `output_types` and `output_shapes` properties of either
# `training_dataset` or `validation_dataset` here, because they have
# identical structure.
handle = tf.placeholder(tf.string, shape=[])
iterator = tf.data.Iterator.from_string_handle(
    handle, training_dataset.output_types, training_dataset.output_shapes)
next_element = iterator.get_next()

# You can use feedable iterators with a variety of different kinds of iterator
training_iterator = training_dataset.make_one_shot_iterator()
validation_iterator = validation_dataset.make_initializable_iterator()

with tf.Session() as sess:
    # The `Iterator.string_handle()` method returns a tensor that can be evaluated
    # and used to feed the `handle` placeholder.
    training_handle = sess.run(training_iterator.string_handle())
    validation_handle = sess.run(validation_iterator.string_handle())
    # Loop forever, alternating between training and validation.
    for i in range(5):
        print("######################## epoch", i)
        i += 1
        # Run 10 steps using the training dataset. Note that the training dataset is
        # 2 * the original set, i.e. we run 2 epochs (see .repeat() argument), and we resume from where
        # we left off in the previous `while` loop iteration.
        for _ in range(10):
            nel = sess.run(next_element, feed_dict={handle: training_handle})
            print("train: ", type(nel), nel)

        # Run one pass over the validation dataset.
        sess.run(validation_iterator.initializer)
        for _ in range(5):
            nel = sess.run(next_element, feed_dict={handle: validation_handle})
            print("valid: ", type(nel), nel)

########################  0
train:  <class 'numpy.int64'> 0
train:  <class 'numpy.int64'> 1
train:  <class 'numpy.int64'> 2
train:  <class 'numpy.int64'> 3
train:  <class 'numpy.int64'> 4
train:  <class 'numpy.int64'> 5
train:  <class 'numpy.int64'> 6
train:  <class 'numpy.int64'> 7
train:  <class 'numpy.int64'> 8
train:  <class 'numpy.int64'> 9
valid:  <class 'numpy.int64'> 0
valid:  <class 'numpy.int64'> 1
valid:  <class 'numpy.int64'> 2
valid:  <class 'numpy.int64'> 3
valid:  <class 'numpy.int64'> 4
########################  1
train:  <class 'numpy.int64'> 10
train:  <class 'numpy.int64'> 11
train:  <class 'numpy.int64'> 12
train:  <class 'numpy.int64'> 13
train:  <class 'numpy.int64'> 14
train:  <class 'numpy.int64'> 15
train:  <class 'numpy.int64'> 16
train:  <class 'numpy.int64'> 17
train:  <class 'numpy.int64'> 18
train:  <class 'numpy.int64'> 19
valid:  <class 'numpy.int64'> 0
valid:  <class 'numpy.int64'> 1
valid:  <class 'numpy.int64'> 2
valid:  <class 'numpy.int64'> 3
valid:

### Function form

In [15]:
def input_fn():
    # Define training and validation datasets with the same structure.
    training_dataset = tf.data.Dataset.range(10000000).repeat(2)
    validation_dataset = tf.data.Dataset.range(5000000).repeat(2)

    # A feedable iterator is defined by a handle placeholder and its structure. We
    # could use the `output_types` and `output_shapes` properties of either
    # `training_dataset` or `validation_dataset` here, because they have
    # identical structure.
    handle = tf.placeholder(tf.string, shape=[])
    iterator = tf.data.Iterator.from_string_handle(
        handle, training_dataset.output_types, training_dataset.output_shapes)
    next_element = iterator.get_next()

    # You can use feedable iterators with a variety of different kinds of iterator
    training_iterator = training_dataset.make_one_shot_iterator()
    validation_iterator = validation_dataset.make_initializable_iterator()
    return next_element, handle, training_iterator, validation_iterator

In [22]:
next_element, handle, training_iterator, validation_iterator=input_fn()

sess=tf.Session()
# The `Iterator.string_handle()` method returns a tensor that can be evaluated
# and used to feed the `handle` placeholder.
training_handle = sess.run(training_iterator.string_handle())
validation_handle = sess.run(validation_iterator.string_handle())

for _ in range(5):
    nel = sess.run(next_element, feed_dict={handle: training_handle})
    print("train: ", type(nel), nel)

sess.close()

train:  <class 'numpy.int64'> 0
train:  <class 'numpy.int64'> 1
train:  <class 'numpy.int64'> 2
train:  <class 'numpy.int64'> 3
train:  <class 'numpy.int64'> 4


In [ ]:
with tf.Session() as sess:
    # The `Iterator.string_handle()` method returns a tensor that can be evaluated
    # and used to feed the `handle` placeholder.
    training_handle = sess.run(training_iterator.string_handle())
    validation_handle = sess.run(validation_iterator.string_handle())
    # Loop forever, alternating between training and validation.
    for i in range(5):
        print("######################## epoch", i)
        i += 1
        # Run 10 steps using the training dataset. Note that the training dataset is
        # 2 * the original set, i.e. we run 2 epochs (see .repeat() argument), and we resume from where
        # we left off in the previous `while` loop iteration.
        for _ in range(10):
            nel = sess.run(next_element, feed_dict={handle: training_handle})
            print("train: ", type(nel), nel)

        # Run one pass over the validation dataset.
        sess.run(validation_iterator.initializer)
        for _ in range(5):
            nel = sess.run(next_element, feed_dict={handle: validation_handle})
            print("valid: ", type(nel), nel)